In [ ]:
# Скачиваем и устанавливаем Miniconda
!wget -O miniconda.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x miniconda.sh
!bash ./miniconda.sh -b -f -p /usr/local

# Принимаем Terms of Service для стандартных каналов
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
!conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Создаём отдельное окружение splatam с Python 3.10
!conda create -y -n splatam python=3.10

# Инициализируем conda (для автозагрузки, но нам это не критично)
!conda init

# Перезапускаем среду выполнения (обязательно!)
import os
os.kill(os.getpid(), 9)

--2026-02-22 11:54:16--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156772981 (150M) [application/octet-stream]
Saving to: ‘miniconda.sh’

miniconda.sh        100%[===================>] 149.51M   182MB/s    in 0.8s    

2026-02-22 11:54:17 (182 MB/s) - ‘miniconda.sh’ saved [156772981/156772981]

PREFIX=/usr/local
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points t

In [ ]:
# Проверяем версию Python внутри окружения (должно быть 3.10)
!conda run -n splatam python --version

# Клонируем репозиторий SplaTAM (если папка уже есть, она будет удалена)
!rm -rf SplaTAM
!git clone https://github.com/yaneegoist/SplaTAM.git
%cd SplaTAM

# Исправляем подмодуль: заменяем SSH на HTTPS
!sed -i 's/git@github.com:/https:\/\/github.com\//' .gitmodules
!git submodule sync
!git submodule update --init --recursive

# Устанавливаем PyTorch 2.3.0 (совместим с CUDA 12.1 в Colab)
!conda run -n splatam pip install torch==2.3.0 torchvision==0.18.0 --index-url https://download.pytorch.org/whl/cu121

# Исправляем ссылку в requirements.txt на рабочий репозиторий diff-gaussian-rasterization-w-depth
# (заменяем старый нерабочий путь на корневой URL репозитория)
!sed -i 's|git+https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git@cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110|git+https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git|' requirements.txt

# Устанавливаем все зависимости (включая исправленный модуль)
!conda run -n splatam pip install -r requirements.txt

# Для надёжности можно также выполнить отдельную установку модуля через setup.py (опционально)
!git clone https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git
%cd diff-gaussian-rasterization-w-depth
!conda run -n splatam python setup.py install
%cd ..

# Проверяем, что модуль успешно загружается
!conda run -n splatam python -c "from diff_gaussian_rasterization import _C; print('Модуль загружен успешно!')"

Python 3.10.19

Cloning into 'SplaTAM'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 286 (delta 158), reused 158 (delta 130), pack-reused 42 (from 1)
Receiving objects: 100% (286/286), 14.37 MiB | 31.23 MiB/s, done.
Resolving deltas: 100% (161/161), done.
/content/SplaTAM
Submodule 'diff-gaussian-rasterization-w-depth.git' (https://github.com/JonathonLuiten/diff-gaussian-rasterization-w-depth.git) registered for path 'diff-gaussian-rasterization-w-depth.git'
Cloning into '/content/SplaTAM/diff-gaussian-rasterization-w-depth.git'...
Submodule path 'diff-gaussian-rasterization-w-depth.git': checked out 'cb65e4b86bc3bd8ed42174b72a62e8d3a3a71110'
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 30.3 MB/s  0:00:13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!git pull

remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 2 (delta 1), reused 1 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (2/2), 226 bytes | 226.00 KiB/s, done.
From https://github.com/yaneegoist/SplaTAM
   fdbf377..7d4c411  main       -> origin/main
Updating fdbf377..7d4c411
Fast-forward
 123.txt | 0
 1 file changed, 0 insertions(+), 0 deletions(-)
 delete mode 100644 123.txt


In [ ]:
!conda run -n splatam bash bash_scripts/download_tum.sh

Выходные данные были обрезаны до нескольких последних строк (5000).
 86150K .......... .......... .......... .......... .......... 25%  278M 6s
 86200K .......... .......... .......... .......... .......... 25%  209M 6s
 86250K .......... .......... .......... .......... .......... 25%  275M 6s
 86300K .......... .......... .......... .......... .......... 25%  265M 6s
 86350K .......... .......... .......... .......... .......... 25%  274M 6s
 86400K .......... .......... .......... .......... .......... 25% 2.85M 6s
 86450K .......... .......... .......... .......... .......... 25% 15.6M 6s
 86500K .......... .......... .......... .......... .......... 25%  194M 6s
 86550K .......... .......... .......... .......... .......... 25%  297M 6s
 86600K .......... .......... .......... .......... .......... 25%  211M 6s
 86650K .......... .......... .......... .......... .......... 25%  307M 6s
 86700K .......... .......... .......... .......... .......... 25%  288M 6s
 86750K .......... .

In [ ]:
!conda run -n splatam pip install opencv-python pytorch-msssim torchmetrics matplotlib kornia lpips wandb natsort pyyaml imageio tqdm open3d==0.19.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 16.4 MB/s  0:00:18
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 58.5 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 63.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 129.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 141.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 114.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 770.3/770.3 kB 53.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 134.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 172.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 60.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 122.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.8/831.8 kB 57.4 MB/s  0:00:00
   ━━━━━

In [ ]:
!conda run -n splatam pip install numpy==1.26.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 55.8 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.



In [ ]:
!sed -i 's/num_frames=100/num_frames=50/' configs/tum/splatam.py
!cat configs/tum/splatam.py | grep num_frames

        num_frames=50,


In [ ]:
!git pull origin add-quality

From https://github.com/yaneegoist/SplaTAM
 * branch            add-quality -> FETCH_HEAD
Already up to date.


In [ ]:
!sed -i 's/num_frames=1/num_frames=150/' configs/tum/splatam.py
!cat configs/tum/splatam.py | grep num_frames

        num_frames=150,


In [ ]:
!sed -i 's/eval_every=500/eval_every=5/' configs/tum/splatam.py
!cat configs/tum/splatam.py | grep eval_every

    eval_every=5, # Evaluate every nth frame (at end of SLAM)


In [ ]:
!sed -i 's/use_wandb=True/use_wandb=False/' configs/tum/splatam.py
!conda run -n splatam python scripts/splatam.py configs/tum/splatam.py

System Paths:
/content/SplaTAM
/content/SplaTAM/scripts
/env/python
/usr/local/envs/splatam/lib/python310.zip
/usr/local/envs/splatam/lib/python3.10
/usr/local/envs/splatam/lib/python3.10/lib-dynload
/usr/local/envs/splatam/lib/python3.10/site-packages
Seed set to: 0 (type: <class 'int'>)
Loaded Config:
{'workdir': './experiments/TUM', 'run_name': 'freiburg1_desk_seed0', 'seed': 0, 'primary_device': 'cuda:0', 'map_every': 1, 'keyframe_every': 5, 'mapping_window_size': 20, 'report_global_progress_every': 500, 'eval_every': 5, 'scene_radius_depth_ratio': 2, 'mean_sq_dist_method': 'projective', 'gaussian_distribution': 'isotropic', 'report_iter_progress': False, 'load_checkpoint': False, 'checkpoint_time_idx': 0, 'save_checkpoints': False, 'checkpoint_interval': 100, 'use_wandb': False, 'wandb': {'entity': 'theairlab', 'project': 'SplaTAM', 'group': 'TUM', 'name': 'freiburg1_desk_seed0', 'save_qual': False, 'eval_save_qual': True}, 'data': {'basedir': './data/TUM_RGBD', 'gradslam_data_cfg

In [ ]:
!sed -i 's/num_frames=-1/num_frames=150/' configs/tumtest/splatam.py
!cat configs/tumtest/splatam.py | grep num_frames

        num_frames=150,


In [ ]:
!sed -i 's/use_wandb=True/use_wandb=False/' configs/tumtest/splatam.py
!conda run -n splatam python scripts/splatamtest.py configs/tumtest/splatam.py

System Paths:
/content/SplaTAM
/content/SplaTAM/scripts
/env/python
/usr/local/envs/splatam/lib/python310.zip
/usr/local/envs/splatam/lib/python3.10
/usr/local/envs/splatam/lib/python3.10/lib-dynload
/usr/local/envs/splatam/lib/python3.10/site-packages
Seed set to: 0 (type: <class 'int'>)
Loaded Config:
{'workdir': './experiments/TUM', 'run_name': 'freiburg1_desk_seed0', 'seed': 0, 'primary_device': 'cuda:0', 'map_every': 1, 'keyframe_every': 5, 'mapping_window_size': 20, 'report_global_progress_every': 500, 'eval_every': 5, 'scene_radius_depth_ratio': 2, 'mean_sq_dist_method': 'projective', 'gaussian_distribution': 'isotropic', 'report_iter_progress': False, 'load_checkpoint': False, 'checkpoint_time_idx': 0, 'save_checkpoints': False, 'checkpoint_interval': 100, 'use_wandb': False, 'new_submap_every': 10, 'submap_overlap_frames': 5, 'wandb': {'entity': 'theairlab', 'project': 'SplaTAM', 'group': 'TUM', 'name': 'freiburg1_desk_seed0', 'save_qual': False, 'eval_save_qual': True}, 'data